This file is intended to test setting up Binder.

In [48]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import getall as get

## User input values:

In [32]:
mineral='olivine'   #Planet composition
Mpl=1.0             #Planet mass in Me
Rpl=1.0             #Planet radius in Re
CMF=0.33            #Core mass fraction
CRF=0.5437          #Core radius fraction
Qpl=1.0             #Relative heat production per kg mantle, vs Earth
Ts=300.0            #Surface temperature in K
Tp=1923.0           #starting mantle potential temperature in K
t=0.0
#U238,U235,Th232,K40=1.0,1.0,1.0,1.0    #Relative abundance of radiogenics, vs earth

#### Retrieve unchanging and/or initial properties based on user inputs.

In [27]:
c1,Ev,visc0=get.TdepVisc(mineral)
Mp,Mc,Rp,Rc=get.SIunits(Mpl,CMF,Rpl,CRF)
d,Vm,SA,pm,g=get.build(Mp,Mc,Rp,Rc)
alpha,cp,k=get.thermals(mineral,Tp)